### 基于维特比算法来优化上述流程

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 根据词典，输入的句子和 word_prob来创建带权重的有向图（Directed Graph） 参考：课程内容
有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率已经给出（存放在word_prob）。
注意：思考用什么方式来存储这种有向图比较合适？ 不一定只有一种方式来存储这种结构。 

#### Step 2: 编写维特比算法（viterebi）算法来找出其中最好的PATH， 也就是最好的句子切分
具体算法参考课程中讲过的内容

#### Step 3: 返回结果
跟PART 1.1的要求一致

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "./data/综合类中文词库.xlsx"
data_frame = pd.read_excel(path, header = None)

In [3]:
dic_word_list = data_frame[data_frame.columns[0]].tolist()

In [4]:
dic_words =  dic_word_list   # 保存词典库中读取的单词

# 以下是每一个单词出现的概率。为了问题的简化，我们只列出了一小部分单词的概率。 在这里没有出现的的单词但是出现在词典里的，统一把概率设置成为0.00001
# 比如 p("学院")=p("概率")=...0.00001

word_prob = {"北京":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.01,"真好啊":0.02, 
             "今":0.01,"今天":0.07,"课程":0.06,"内容":0.06,"有":0.05,"很":0.03,"很有":0.04,"意思":0.06,"有意思":0.005,"课":0.01,
             "程":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}

In [5]:
for key, value in word_prob.items():
    if key not in dic_words:
        dic_words.append(key)

In [6]:
for item in dic_words:
    word_prob.setdefault(item, 0.00001)

In [7]:
def set_graph_back_set_log(input_str, word_prob = word_prob):
    '''
    逆邻接表
    未计算log值，找到后计算对应log值
    '''
    str_len = len(input_str)
    master_list = [{} for _ in range(str_len + 1)] # 初始化含有定点的主列表
    window = 1 # 初始化窗口大小
    start_position = 0 # 窗口起始位置
    while(window <= str_len):
        end_position = start_position + window - 1 # 窗口结束为止
        split_str = input_str[start_position:end_position] # 当前查找的字符串
        log_value = word_prob.get(split_str)# 找到当前字符串对应的log值
        if(None != log_value): # 值存在则插入字典
            master_list[end_position][start_position] = -np.log(log_value)
        if(str_len > end_position): start_position += 1# 向后一位进行检测
        else:
            window += 1 # 增大窗口大小
            start_position = 0
        
    return master_list

In [8]:
def get_total_weight(list_input, index):
    if(list_input[index][0] == 0 and list_input[index][1] == 1): return 0
    else:
        return list_input[index][0] + get_total_weight(list_input, list_input[index][1])
def calc_min_weight_path_back(master_list):
    '''
    逆邻接表求最短路径
    '''
    len_master_list = len(master_list)
#     tmp_min_list = [[None, None]] * len_master_list # 初始化给定长度的数组保存最短路径
    
    tmp_min_list = [[None] * 2 for row in range(len_master_list)] # 初始化给定长度的数组保存最短路径
    tmp_min_list[0] = [0, 1] # 初始化第一个节点默认代表 最小值、来源
    for index in range(1, len_master_list):
        for key, value in master_list[index].items():
            if(tmp_min_list[index][0] == None):
                tmp_min_list[index][0] = value
                tmp_min_list[index][1] = key
            else:
                tmp_sum1 = get_total_weight(tmp_min_list, index) # 原来的结果
                tmp_sum2 = get_total_weight(tmp_min_list, tmp_min_list[key][1]) + value # 待插入的结果
                if(tmp_sum2 < tmp_sum1): # 新插入的值更小
                    tmp_min_list[index][0] = value
                    tmp_min_list[index][1] = key
    
    return tmp_min_list

In [9]:
min_path = [1, 0, 0, 2, 3, 3, 5, 5, 5]
min_path[-1]

5

In [10]:
def get_split_by_viterbi(min_path_list, input_str):
    min_path = [ i[1] for i in min_path_list]
    tmp_list =[]
    last_position = len(input_str)    
    position = min_path[-1]
    while(position != 0):
        tmp_list.append(input_str[position:last_position])
        last_position = position
        position = min_path[position]
    tmp_list.append(input_str[position:last_position])
    return tmp_list[::-1]

In [11]:
# 分数（10）

## TODO 请编写word_segment_viterbi函数来实现对输入字符串的分词
def word_segment_viterbi(input_str):
    """
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    
    # 第一步：根据词典，输入的句子，以及给定的unigram概率来创建带权重的有向图（Directed Graph） 参考：课程内容
    graph = set_graph_back_set_log(input_str)
    
    # 第二步： 利用维特比算法来找出最好的PATH， 这个PATH是P(sentence)最大或者 -log P(sentence)最小的PATH。
    
    min_path_list = calc_min_weight_path_back(graph)
    
    # 第三步： 根据最好的PATH, 返回最好的切分
    
    best_segment = get_split_by_viterbi(min_path_list, input_str)

    return best_segment      

In [12]:
print (word_segment_viterbi("北京的天气真好啊"))
print (word_segment_viterbi("今天的课程内容很有意思"))
print (word_segment_viterbi("经常有意见分歧"))

['北京', '的', '天气', '真好啊']
['今天', '的', '课程', '内容', '很', '有意思']
['经常', '有意见', '分歧']
